In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn import datasets
from IPython.display import display, HTML

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [10]:
batter = pd.read_csv('/content/gdrive/My Drive/baseball/data1/teambat2020.csv',encoding= 'CP949')
batter.head()

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,PA,AB,RBI,RUN,HIT,H2,H3,HR,SB,CS,SH,SF,BB,IB,HP,KK,GD,ERR,LOB,P_HRA_RT,P_AB_CN,P_HIT_CN
0,20200505HHSK0,20200505,SK,HH,0,B,30,29,0,0,2,0,0,0,0,0,0,0,1,0,0,2,0,0,3,0.000000,1,0
1,20200505HHSK0,20200505,HH,SK,0,T,35,31,3,3,8,3,0,0,0,1,2,0,2,0,0,5,1,0,5,0.250000,8,2
2,20200505LTKT0,20200505,KT,LT,0,B,35,30,2,2,5,1,0,1,0,1,0,0,5,0,0,8,1,0,6,0.000000,4,0
3,20200505LTKT0,20200505,LT,KT,0,T,37,32,7,7,7,1,0,2,0,0,0,1,4,0,0,9,1,1,3,0.666667,3,2
4,20200505NCSS0,20200505,SS,NC,0,B,36,30,0,0,3,0,0,0,0,1,0,0,6,0,0,8,0,0,9,0.000000,5,0


In [11]:
batter = batter.drop(['HEADER_NO','GDAY_DS','TB_SC','P_HRA_RT','P_AB_CN','P_HIT_CN','LOB','ERR'], axis=1)
batter.head()

,G_ID,T_ID,VS_T_ID,PA,AB,RBI,RUN,HIT,H2,H3,HR,SB,CS,SH,SF,BB,IB,HP,KK,GD
0,20200505HHSK0,SK,HH,30,29,0,0,2,0,0,0,0,0,0,0,1,0,0,2,0
1,20200505HHSK0,HH,SK,35,31,3,3,8,3,0,0,0,1,2,0,2,0,0,5,1
2,20200505LTKT0,KT,LT,35,30,2,2,5,1,0,1,0,1,0,0,5,0,0,8,1
3,20200505LTKT0,LT,KT,37,32,7,7,7,1,0,2,0,0,0,1,4,0,0,9,1
4,20200505NCSS0,SS,NC,36,30,0,0,3,0,0,0,0,1,0,0,6,0,0,8,0


### 2020년 SK 타선의 성적 (팀 전체로 봤을 떄)

In [13]:
sk_batter2020 = batter[batter.T_ID=='SK']
sk_batter2020

,G_ID,T_ID,VS_T_ID,PA,AB,RBI,RUN,HIT,H2,H3,HR,SB,CS,SH,SF,BB,IB,HP,KK,GD
0,20200505HHSK0,SK,HH,30,29,0,0,2,0,0,0,0,0,0,0,1,0,0,2,0
10,20200506HHSK0,SK,HH,37,33,5,5,9,3,0,2,3,0,1,0,3,0,0,8,0
20,20200507HHSK0,SK,HH,45,39,3,4,15,2,2,0,0,0,0,0,4,0,2,8,3
39,20200508SKLT0,SK,LT,45,40,8,8,13,3,1,2,4,0,1,2,2,0,0,7,0
53,20200510SKLT0,SK,LT,29,29,0,0,3,1,0,0,0,0,0,0,0,0,0,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,20200715SKOB0,SK,OB,37,35,2,2,10,0,0,0,1,0,0,1,1,0,0,10,0
609,20200716SKOB0,SK,OB,37,34,2,2,7,0,0,1,5,0,0,0,3,0,0,6,2
618,20200717WOSK0,SK,WO,43,35,11,12,13,2,0,2,0,1,0,1,7,0,0,7,0
628,20200718WOSK0,SK,WO,34,32,1,1,5,0,0,1,0,0,0,0,2,0,0,8,0


#### 2020년 sk의 평균 득점

In [18]:
np.mean(sk_batter2020.RUN)

3.876923076923077

#### 출루율

In [23]:
sk_obp = (sk_batter2020.HIT + sk_batter2020.H2 + sk_batter2020.H3 + sk_batter2020.HR + sk_batter2020.BB + sk_batter2020.HP + sk_batter2020.IB)/sk_batter2020.PA
sk_obp

0      0.100000
10     0.459459
20     0.555556
39     0.466667
53     0.137931
         ...   
599    0.297297
609    0.297297
618    0.558140
628    0.235294
638    0.351351
Length: 65, dtype: float64

In [24]:
sk_batter2020['OBP'] = sk_obp
sk_batter2020.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,G_ID,T_ID,VS_T_ID,PA,AB,RBI,RUN,HIT,H2,H3,HR,SB,CS,SH,SF,BB,IB,HP,KK,GD,OBP
0,20200505HHSK0,SK,HH,30,29,0,0,2,0,0,0,0,0,0,0,1,0,0,2,0,0.100000
10,20200506HHSK0,SK,HH,37,33,5,5,9,3,0,2,3,0,1,0,3,0,0,8,0,0.459459
20,20200507HHSK0,SK,HH,45,39,3,4,15,2,2,0,0,0,0,0,4,0,2,8,3,0.555556
39,20200508SKLT0,SK,LT,45,40,8,8,13,3,1,2,4,0,1,2,2,0,0,7,0,0.466667
53,20200510SKLT0,SK,LT,29,29,0,0,3,1,0,0,0,0,0,0,0,0,0,12,1,0.137931


#### 장타율

In [25]:
sk_slg = (sk_batter2020.HIT + 2*sk_batter2020.H2 + 3*sk_batter2020.H3 + 4*sk_batter2020.HR )/sk_batter2020.AB
sk_slg

0      0.068966
10     0.696970
20     0.641026
39     0.750000
53     0.172414
         ...   
599    0.285714
609    0.323529
618    0.714286
628    0.281250
638    0.200000
Length: 65, dtype: float64

In [26]:
sk_batter2020['SLG'] = sk_slg
sk_batter2020.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,G_ID,T_ID,VS_T_ID,PA,AB,RBI,RUN,HIT,H2,H3,HR,SB,CS,SH,SF,BB,IB,HP,KK,GD,OBP,SLG
0,20200505HHSK0,SK,HH,30,29,0,0,2,0,0,0,0,0,0,0,1,0,0,2,0,0.100000,0.068966
10,20200506HHSK0,SK,HH,37,33,5,5,9,3,0,2,3,0,1,0,3,0,0,8,0,0.459459,0.696970
20,20200507HHSK0,SK,HH,45,39,3,4,15,2,2,0,0,0,0,0,4,0,2,8,3,0.555556,0.641026
39,20200508SKLT0,SK,LT,45,40,8,8,13,3,1,2,4,0,1,2,2,0,0,7,0,0.466667,0.750000
53,20200510SKLT0,SK,LT,29,29,0,0,3,1,0,0,0,0,0,0,0,0,0,12,1,0.137931,0.172414


#### OPS

In [27]:
sk_ops = sk_batter2020.OBP + sk_batter2020.SLG
sk_batter2020['OPS'] = sk_ops
sk_batter2020.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,G_ID,T_ID,VS_T_ID,PA,AB,RBI,RUN,HIT,H2,H3,HR,SB,CS,SH,SF,BB,IB,HP,KK,GD,OBP,SLG,OPS
0,20200505HHSK0,SK,HH,30,29,0,0,2,0,0,0,0,0,0,0,1,0,0,2,0,0.100000,0.068966,0.168966
10,20200506HHSK0,SK,HH,37,33,5,5,9,3,0,2,3,0,1,0,3,0,0,8,0,0.459459,0.696970,1.156429
20,20200507HHSK0,SK,HH,45,39,3,4,15,2,2,0,0,0,0,0,4,0,2,8,3,0.555556,0.641026,1.196581
39,20200508SKLT0,SK,LT,45,40,8,8,13,3,1,2,4,0,1,2,2,0,0,7,0,0.466667,0.750000,1.216667
53,20200510SKLT0,SK,LT,29,29,0,0,3,1,0,0,0,0,0,0,0,0,0,12,1,0.137931,0.172414,0.310345


In [28]:
sk_ops

0      0.168966
10     1.156429
20     1.196581
39     1.216667
53     0.310345
         ...   
599    0.583012
609    0.620827
618    1.272425
628    0.516544
638    0.551351
Length: 65, dtype: float64

#### GPA

In [29]:
sk_gpa = 1.8*sk_batter2020.OBP + sk_batter2020.SLG
sk_batter2020['GPA'] = sk_gpa
sk_batter2020.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,G_ID,T_ID,VS_T_ID,PA,AB,RBI,RUN,HIT,H2,H3,HR,SB,CS,SH,SF,BB,IB,HP,KK,GD,OBP,SLG,OPS,GPA
0,20200505HHSK0,SK,HH,30,29,0,0,2,0,0,0,0,0,0,0,1,0,0,2,0,0.100000,0.068966,0.168966,0.248966
10,20200506HHSK0,SK,HH,37,33,5,5,9,3,0,2,3,0,1,0,3,0,0,8,0,0.459459,0.696970,1.156429,1.523997
20,20200507HHSK0,SK,HH,45,39,3,4,15,2,2,0,0,0,0,0,4,0,2,8,3,0.555556,0.641026,1.196581,1.641026
39,20200508SKLT0,SK,LT,45,40,8,8,13,3,1,2,4,0,1,2,2,0,0,7,0,0.466667,0.750000,1.216667,1.590000
53,20200510SKLT0,SK,LT,29,29,0,0,3,1,0,0,0,0,0,0,0,0,0,12,1,0.137931,0.172414,0.310345,0.420690


In [30]:
np.mean(sk_batter2020.OPS)

0.7703668556360039